In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import concurrent.futures

# Global options
N_repeat = 1000  # Number of simulations to run
precision = 1e-10
max_attempts = 10
max_time = 5
switch_competitive = 0
t_fit_start = 500
t_fit_end = 600

# ODE options
T_preturb = 5e2
T_posturb = 5e2
N_points_pre = int(T_preturb * 100)
N_points_post = int(T_posturb * 100)
t_pre = np.linspace(0., T_preturb, N_points_pre)
t_post = np.linspace(T_preturb, T_preturb + T_posturb, N_points_post)

# Parameter settings
N_species = 5
Connectivity = 1.0
dilution_coeff = 0.1
N_ini = 0.2
alpha_mean = 0.0
alpha_std = 0.25
exponent_intra = 1.0    # Alpha
exponent_inter = 1.0    # Beta

# Define the ODE system
def derivative(t, N, r, a, K, exponent_intra, exponent_inter):
    N = np.array(N)  # Ensure N is a NumPy array
    N = np.clip(N, 0, None)  # Removing negative population density if happens
    sum_aN_off_diag = np.dot(a - np.diag(np.diag(a)), N ** exponent_inter)
    sum_aN_diag = np.diag(a) * np.where(N > 1e-10, N ** exponent_intra, 0)
    sum_aN = np.real(sum_aN_off_diag + np.sign(exponent_intra) * sum_aN_diag)
    dotN = N * (np.sign(exponent_intra) * r - sum_aN) / K
    return dotN

# Generate interaction matrix
def generate_interaction_matrix(num_species, connectivity, mean, sd):
    a = np.zeros((num_species, num_species))
    np.fill_diagonal(a, 1)
    for i in range(num_species):
        for j in range(num_species):
            if i != j and np.random.random() < connectivity:
                a[i, j] = np.random.normal(loc=mean, scale=sd)
    return a

# Timeout handler using threading
class ODESolverWithTimeout:
    def __init__(self, fun, t_span, y0, args=(), max_time=5, t_eval=None, precision=1e-10, max_step=0.1):
        self.fun = fun
        self.t_span = t_span
        self.y0 = y0
        self.args = args
        self.max_time = max_time
        self.t_eval = t_eval
        self.precision = precision
        self.max_step = max_step
        self.result = None
        self.thread = None

    def solve(self):
        self.thread = threading.Thread(target=self._solve_ode)
        self.thread.start()
        self.thread.join(timeout=self.max_time)
        if self.thread.is_alive():
            print("Solver timed out")
            self.result = None
        return self.result

    def _solve_ode(self):
        try:
            sol = solve_ivp(self.fun, self.t_span, self.y0, args=self.args, t_eval=self.t_eval, method='BDF', atol=self.precision, rtol=self.precision, max_step=self.max_step)
            self.result = sol
        except Exception as e:
            print(f"Solver failed: {e}")
            self.result = None

# Model function for Model 2: dN_i/dt = N_i * (r_i - N_i^alpha)
def model2(N, r, alpha):
    N = np.clip(N, 1e-10, None)
    return N * (r - N**alpha)

# Function to run a single simulation and add noise
def run_simulation(repeat):
    np.random.seed()  # Ensure a new random seed is generated for each simulation
    noise_levels = [0, 0.001, 0.01, 0.1]
    alphas_for_simulation = []
    
    for attempt in range(max_attempts):
        # Sampling a parameter combination
        K = np.repeat(1., N_species)
        a = generate_interaction_matrix(N_species, connectivity=1.0, mean=0.0, sd=0.25)
        if switch_competitive == 1:
            a = np.abs(a)
        r = np.repeat(1., N_species)

        # Stage 1: Solve the ODE system
        N_initial = np.repeat(0.2, N_species)
        solver1 = ODESolverWithTimeout(derivative, [0, T_preturb], y0=N_initial, args=(r, a, K, 1.0, 1.0), t_eval=t_pre, precision=1e-10, max_time=max_time)
        solved1 = solver1.solve()
        if solved1 is not None:
            N_pre = solved1.y
            N_mid = N_pre[:, -1]
            N_perturb = N_mid * dilution_coeff
            if not(np.any(np.isnan(N_perturb)) or np.any(np.isinf(N_perturb))):
                # Stage 2: Solve the system again after perturbation
                solver2 = ODESolverWithTimeout(derivative, [T_preturb, T_preturb + T_posturb], y0=N_perturb, args=(r, a, K, 1.0, 1.0), t_eval=t_post, precision=1e-10, max_time=max_time)
                solved2 = solver2.solve()
                if solved2 is not None and solved2.success:
                    N_post = solved2.y

                    # Apply different noise levels and fit for each
                    for noise_level in noise_levels:
                        # Add noise
                        N_noisy = N_post + noise_level * np.random.normal(0, 1, N_post.shape)

                        # Model fitting for each species
                        alpha_fits = []
                        for i in range(N_species):
                            N_i = N_noisy[i, :]
                            t = t_post

                            # Compute numerical derivative dN_i/dt
                            dN_i_dt = np.gradient(N_i, t)

                            # Create time mask for the fitting interval
                            time_mask = (t >= t_fit_start) & (t <= t_fit_end)

                            # Extract valid data points
                            N_i_valid = N_i[time_mask]
                            dN_i_dt_valid = dN_i_dt[time_mask]

                            # Ensure there are enough data points to fit
                            if len(N_i_valid) < 5:
                                continue

                            # Fit Model 2: dN_i/dt = N_i * (r_i - N_i^alpha)
                            try:
                                popt2, _ = curve_fit(
                                    model2, N_i_valid, dN_i_dt_valid, p0=[1.0, 1.0], maxfev=10000
                                )
                                alpha_fits.append(popt2[1])  # Store the alpha parameter
                            except RuntimeError as e:
                                print(f"Model 2 fitting failed for species {i} with noise {noise_level}: {e}")
                                continue
                        
                        # Average alpha for this noise level
                        alphas_for_simulation.append(np.mean(alpha_fits) if alpha_fits else np.nan)
            break  # Exit after successful run
    
    return alphas_for_simulation

In [2]:
import threading
import random
import concurrent.futures
# Number of threads to use
num_threads = 32  # Adjust this based on your system's capacity
fitted_alphas = np.zeros((N_repeat, 4))  # Store alphas for each noise level (0, 0.01, 0.1, 1.0)

# Parallel execution using ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as executor:
    # Submit each simulation task to the pool
    results = list(executor.map(run_simulation, range(N_repeat)))

# Combine results from all simulations
for i, res in enumerate(results):
    fitted_alphas[i, :] = res  # Store results for each simulation

# Save the data to a CSV file
np.savetxt('fitted_alphas.csv', fitted_alphas, delimiter=',', header="NoNoise,Noise_0.001,Noise_0.01,Noise_0.1", comments='')


/tmp/ipykernel_4096845/3425048459.py:87: RuntimeWarning: overflow encountered in power
  return N * (r - N**alpha)
/tmp/ipykernel_4096845/3425048459.py:87: RuntimeWarning: overflow encountered in power
  return N * (r - N**alpha)
/home/biology/worc5321/.local/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_4096845/3425048459.py:87: RuntimeWarning: overflow encountered in power
  return N * (r - N**alpha)
/tmp/ipykernel_4096845/3425048459.py:87: RuntimeWarning: overflow encountered in power
  return N * (r - N**alpha)
/home/biology/worc5321/.local/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_4096845/3425048459.py:87: RuntimeWarning: overflow encounte